# Topics in Computer Science - Bitcoin: Programming the Future of Money - ITCS 4010 & 5010 - Spring 2025 - UNC Charlotte

# Homework 7 - Sending a Testnet Transaction, Lightning Network (65 Points)

Names of Student: Hritika Kucheriya

Charlotte ID: 801456028

List of students collaborated with:

# <font color="blue"> Submission instructions</font>

1. Click the Save button at the top of the Jupyter Notebook.
2. Please make sure to have entered your name above.
3. Select Cell -> All Output -> Clear. This will clear all the outputs from all cells (but will keep the content of all cells). 
4. Select Cell -> Run All. This will run all the cells in order, and will take several minutes.
5. Once you've rerun everything, select File -> Download as -> PDF via LaTeX and download a PDF version showing the code and the output of all cells, and save it in the same folder that contains the notebook file.
6. Look at the PDF file and make sure all your solutions are there, displayed correctly.
7. Submit **both** your PDF and the notebook file .ipynb on Gradescope.
8. Make sure your your Gradescope submission contains the correct files by downloading it after posting it on Gradescope.

### Make sure you have the output of all cells in the PDF version.

In [1]:
import requests
from helper import little_endian_to_int, hash256
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
from ecc import PrivateKey
from io import BytesIO

### Exercise 1 (10 Points)

Recall that a [UTXO (Unspent Transaction Output)](https://learnmeabitcoin.com/technical/transaction/utxo/) is a key object in the Bitcoin protocol, represent a denomination of bitcoins that corresponds to one of the outputs of a transaction, but which has not yet been spent. 

For the purpose of this homework, we will represent a UTXO as a tuple, where the first element is the transaction ID (of the transaction of which the UTXO is one of the outputs), and the second element is the output index.

Write a function `BuildTx1to2` that accepts `(UTXO, fromAddress, toAddress)` as inputs and constructs a 1 Input-2 Output transaction. The function should first prompt the user to specify the exact amount (in sats or in BTC) to be sent from `fromAddress` to `toAddress`. Next, it should ask for the transaction fee (in sats) that you want to set for this transaction. The function should then also calculate the correct output amount that corresponds to the second output of the transaction, which corresponds to the "change UTXO" that is going back to the `fromAddress`. You can assume that `fromAddress` and `toAddress` are P2PKH addresses.

Once the output amounts are determined, the function should construct a P2PKH testnet transaction that transfers the specified amount from `fromAddress` to `toAddress` and returns the change amount to `fromAddress`. The function should return a `Tx` class instance (see `tx.py`) displaying the current txID, version, input transactions, output transactions and locktime.

(When using this function, ensure the fee is reasonable to successfully broadcast the transaction to the Testnet4 network.)

**Hints**: Use the classes and functions from `tx.py`, `script.py`, and `helper.py` as needed. Look up how the `TxIn` and `TxOut` classes work in `tx.py`: `TxIn` is able to prepare an "input" part of a transaction, and the `TxOut` class is able to prepare an "output" part of a transaction. You can use the`decode_base58` function from `helper.py` to obtain public key hashes from P2PKH addresses in a 20-byte representation. Consider using  `p2pkh_script` function from `script.py` to obtain the a suitable ScriptPubKey.


In [3]:
#YOUR CODE HERE

def BuildTx1to2(UTXO, fromAddress, toAddress):
    
    
    amount_input = input("Enter amount to send (specify 'sats' or 'BTC' after number, e.g., '50000 sats' or '0.0005 BTC'): ")
    amount_parts = amount_input.split()
    amount = float(amount_parts[0])
    unit = amount_parts[1].lower()
    
    
    if unit == 'btc':
        amount = int(amount * 100_000_000)  
    else:
        amount = int(amount)  
        
    
    fee = int(input("Enter transaction fee (in sats): "))
    
    
    tx_in = TxIn(
        prev_tx=bytes.fromhex(UTXO[0]),
        prev_index=UTXO[1]
    )
    
    
    input_amount = tx_in.value(testnet=True)
    
    
    change_amount = input_amount - amount - fee
    if change_amount < 0:
        raise ValueError("Insufficient funds: input amount is less than output amount + fee")
    
    
    h160_to = decode_base58(toAddress)
    h160_from = decode_base58(fromAddress)
    
    
    tx_out1 = TxOut(
        amount=amount,
        script_pubkey=p2pkh_script(h160_to)
    )
    
    tx_out2 = TxOut(
        amount=change_amount,
        script_pubkey=p2pkh_script(h160_from)
    )
    
    
    tx = Tx(
        version=1,
        tx_ins=[tx_in],
        tx_outs=[tx_out1, tx_out2],
        locktime=0,
        testnet=True
    )
    
    return tx

### Exercise 2 (10 Points)

Create the UTXO tuple using the transaction id and output index from the transaction corresponding to the funding your testnet4 P2PKH address using the Testnet4 faucet you used in Exercise 4 of Homework 6.

**Hint:** You can find the transaction id (TxID) TxID and index from [mempool.space](https://mempool.space/testnet4).

Create a testnet4 transaction that sends one UTXO to the P2PKH address `mpiEesA69LYWvdgRaBFV5DXEcjDGnTPHrp` with an amount of at least $50000$ sats. You can choose the fee at your will, but set it at least at 2000 sats.

For this transaction, **print the transaction object, serialized transaction object and fee.**

#### Note: Submissions without printed outputs may lead to point deductions.

In [19]:

from helper import decode_base58, hash256, hash160
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx, SIGHASH_ALL
from ecc import PrivateKey


my_address = 'mmzyqYxQTao1hrsq6hFEquBbPhhg1qzmxy'
target_address = 'mv4rnyY3Su5gjcDNzbMLKBQkBicCtHUtFB'


prev_tx = bytes.fromhex('e83abbaa2ae837fee57e4520bc24a9d6eb806da9abfc9567f344ea3e32aa2e67')
prev_index = 1


tx_in = TxIn(prev_tx, prev_index)


print("\nEnter amount to send (in sats):")
amount = int(input())
print("Enter transaction fee (in sats):")
fee = int(input())


tx_in_value = tx_in.value(testnet=True)
change_amount = tx_in_value - amount - fee

if change_amount < 0:
    raise ValueError(f"Insufficient funds. UTXO has {tx_in_value} sats, trying to send {amount} with fee {fee}")


h160_recipient = decode_base58(target_address)[1:-4]
script_pubkey_recipient = p2pkh_script(h160_recipient)
tx_out1 = TxOut(amount, script_pubkey_recipient)

h160_sender = decode_base58(my_address)[1:-4]
script_pubkey_change = p2pkh_script(h160_sender)  
tx_out2 = TxOut(change_amount, script_pubkey_change)


tx = Tx(1, [tx_in], [tx_out1, tx_out2], 0, testnet=True)

print(f"\nTransaction created:")
print(f"Input amount: {tx_in_value} sats")
print(f"Output 1 (recipient): {amount} sats")
print(f"Output 2 (change): {change_amount} sats")
print(f"Fee: {fee} sats") 


Enter amount to send (in sats):
Enter transaction fee (in sats):

Transaction created:
Input amount: 500000 sats
Output 1 (recipient): 41000 sats
Output 2 (change): 457000 sats
Fee: 2000 sats


### Exercise 3 (10 Points)

Using the private key you used in Homework 6, sign the input (or inputs, if applicable) of your transaction. Print then the serialized raw (signed) transaction.

(When submitting this part of the homework, make sure that you do not leak your private key to us!)

**Hint:** You may use `sign_input` method of `tx.py` to sign the inputs.

In [20]:
#YOUR CODE HERE


from helper import decode_base58, hash256, hash160
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx, SIGHASH_ALL
from ecc import PrivateKey


my_secret = 1234567890
my_private_key = PrivateKey(my_secret)


my_address = 'mmzyqYxQTao1hrsq6hFEquBbPhhg1qzmxy'
h160 = hash160(my_private_key.point.sec())
raw = b'\x6f' + h160  # 0x6f is the testnet P2PKH version
checksum = hash256(raw)[:4]
addr = decode_base58(my_address)
print(f"Address verification - Expected h160: {addr[1:-4].hex()}, Got h160: {h160.hex()}")


if 'tx' in globals():
    tx.sign_input(0, my_private_key)
    print("\nVerifying signature...")
    print(f"Signature verification result: {tx.verify()}")
    print("\nSigned transaction hex:")
    print(tx.serialize().hex())
else:
    print("Please run Exercise 2 first to create the transaction object 'tx'") 

Address verification - Expected h160: 1db0915ffe1118a3c0c7c39ff6d82a, Got h160: 471db0915ffe1118a3c0c7c39ff6d82ab70f75b2

Verifying signature...
Signature verification result: True

Signed transaction hex:
0100000001672eaa323eea44f36795fcaba96d80ebd6a924bc20457ee5fe37e82aaabb3ae8010000006b483045022100caafcc45f15a3165487ae8c7fc8d0365f336d34cfb02e5d4446dc4db88e0f3af02204f6cdef4d06cec5679ab0b7134166c35a43c7bf958b5d28858a71d84a8db33b00121022b698a0f0a4041b77e63488ad48c23e8e8838dd1fb7520408b121697b782ef22ffffffff0228a00000000000001476a90f9a7abd600c0caa03983a77c8c3df8e88ac28f90600000000001476a90f1db0915ffe1118a3c0c7c39ff6d82a88ac00000000


### Exercise 4 (20 Points)

Submit this transaction to the testnet4 network and make sure that the transaction becomes part of the testnet4 blockchain.

To do this, you have two options:
- The recommended way of doing this is to use Bitcoin Core of version 0.28 or higher, as this version of Bitcoin Core supports testnet4. You can follow [these hints](https://bitcoin.stackexchange.com/questions/124443/how-can-i-enable-testnet4-in-bitcoin-core-28-0) to see how to use Bitcoin Core on testnet4. Downloading the full testnet4 blockchain should not take up more than 9 GB and should be probably be feasible on your personal desktop/laptop computer. You can look for an appropriate RPC command that does the job of submitting the transaction from Exercise 3.
- You can use a third-party API to do the job for you.

**Document the procedure your followed, and print the transaction ID (TxID).**

This exercise will be graded also based on the fact whether the coins actually arrive at the address or not.

In [21]:
import requests

def broadcast_transaction(signed_tx_hex):
    url = "https://mempool.space/testnet4/api/tx"
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=signed_tx_hex, headers=headers)
    
    if response.status_code == 200:
        return response.text.strip()  # Transaction ID if successful
    else:
        return f"Error broadcasting transaction: {response.text}"

# The signed transaction hex from Exercise 3
signed_tx_hex = "0100000001672eaa323eea44f36795fcaba96d80ebd6a924bc20457ee5fe37e82aaabb3ae8010000006b483045022100caafcc45f15a3165487ae8c7fc8d0365f336d34cfb02e5d4446dc4db88e0f3af02204f6cdef4d06cec5679ab0b7134166c35a43c7bf958b5d28858a71d84a8db33b00121022b698a0f0a4041b77e63488ad48c23e8e8838dd1fb7520408b121697b782ef22ffffffff0228a00000000000001476a90f9a7abd600c0caa03983a77c8c3df8e88ac28f90600000000001476a90f1db0915ffe1118a3c0c7c39ff6d82a88ac00000000"

print("Broadcasting transaction to testnet4...")
txid = broadcast_transaction(signed_tx_hex)
print(f"\nTransaction ID (TxID): {txid}")
print(f"\nYou can view your transaction at: https://mempool.space/testnet4/tx/{txid}")

Broadcasting transaction to testnet4...

Transaction ID (TxID): Error broadcasting transaction: sendrawtransaction RPC error: {"code":-26,"message":"scriptpubkey"}

You can view your transaction at: https://mempool.space/testnet4/tx/Error broadcasting transaction: sendrawtransaction RPC error: {"code":-26,"message":"scriptpubkey"}


### Exercise 5 (10 Points)

Assume Alice and Bob have previously created a Lightning Network payment channel with an original balance of 0 satoshis on Alice's side of the channel and a balance of 200,000 satoshis on Bob's side of the channel. After transacting within this payment channel, the current balance is 140,000 satoshis on Alice's side and 60,000 satoshis on Bob's side.

Alice is on holidays and unfortunately, the computer she uses at home to run the Lightning Network node software is offline after a electricity outage. Bob notices that Alice's node is unreachable. He uses this opportunity to publish the original commitment transaction that repays him 200,000 satoshis, in order to steal 140,000 satoshis of Alice's funds.

Assuming that is not able to get her node back online in the next three weeks, what can Alice do (or could have done) to prevent a loss of funds? Explain a mechanism that can prevent her losing her 140,000 satoshis in the above scenario.

(provide your answer here below)

The mechanism that protects Alice's funds in this scenario is called the "breach remedy" or "penalty mechanism" in the Lightning Network. Here's how it works:

1. Every time a new channel state is created (like when the balance changed from 0/200,000 to 140,000/60,000), each party receives:
   - A new commitment transaction
   - A "revocation key" for the previous state

2. When Bob tries to broadcast an old commitment transaction (the original 0/200,000 state), he is actually publishing a transaction that has been "revoked". The revocation key that Alice received allows her to claim ALL the funds in the channel (both her 140,000 sats AND Bob's 60,000 sats) if Bob tries to cheat this way.

3. To protect herself while being offline, Alice could:
   - Run a "watchtower" service that monitors the blockchain for her
   - Give her revocation keys to the watchtower
   - The watchtower can then act on her behalf to claim all funds if Bob broadcasts an old state


### Exercise 6 (5 Points)

We learnt in class that Alice can send funds to Bob in the Lightning Network even if she does not have a direct payment channel to Bob, but if she is indirectly connected to Bob through a "path" of Lightning Network channels, for example, via a pair of channels that include Charlie (one channel between Alice and Charlie, one channel between Charlie and Bob). 

Using your knowledge of the Lightning Network protocol, explain a reason why Charlie would forward any payment between Alice and Bob for a payment in which he is not a recipient.

(provide your answer here below)

Charlie is incentivized to forward payments between Alice and Bob because he earns routing fees for this service. Here's how it works:

1. When Alice wants to send a payment to Bob through Charlie, she includes a small fee that Charlie can collect for forwarding the payment.

2. This routing fee is an economic incentive for Charlie to:
   - Keep his node online and reliable
   - Maintain enough channel liquidity to forward payments
   - Process payments quickly and correctly

3. By operating as a routing node, Charlie can earn passive income from his locked-up capital in the Lightning Network channels. The more well-connected his node is and the more liquidity he provides, the more routing opportunities he has to earn fees.
